# Dialogue and Narrative Coursework - Subtask 1 - Scoring Ground Truths

In [ ]:
## imports
%%capture
!pip install datasets

from datasets import load_dataset, Dataset
import pandas as pd

In [ ]:
def load_doc2dial_dataset(name='dialogue_domain', split='train'):
  cache_dir = "./data_cache"

  return load_dataset(
      "doc2dial",
      name=name,
      split=split,
      ignore_verifications=True,
      cache_dir=cache_dir,
  )

val_data = load_doc2dial_dataset(name="doc2dial_rc", split="validation")

Downloading:   0%|          | 0.00/3.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Reusing dataset doc2dial (./data_cache/doc2dial/doc2dial_rc/1.0.1/765cb4d9af421b599d910080fd61b4a43440c1232693876470ef3245daa5fa4c)


In [ ]:
new_val_data = val_data
new_val_data[:10]

{'answers': [{'answer_start': [1796],
   'text': ['Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage , ']},
  {'answer_start': [2138],
   'text': ['we mail you an insurance inquiry letter to allow you to clear up the problem. ']},
  {'answer_start': [1023],
   'text': ['Learn more about how to change the address on your license and registrations [1 ] ']},
  {'answer_start': [2520],
   'text': ['you won t learn that you have an insurance problem , and we will suspend your registration and license. Make sure you turn in your vehicle s license plates at DMV before you cancel your insurance policy. ']},
  {'answer_start': [1796],
   'text': ['Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insura

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
import ast

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def get_ground_truths(dataset, index):
  instance = dataset[index]
  text = instance["answers"]["text"][0]
  return {"id":instance["id"], "prediction_text":text, "no_answer_probability":0}

In [ ]:
## Example
new_val_data_slice = new_val_data.select(list(range(0, 10)))

for i in range(len(new_val_data_slice)):
  print("QUESTION:", new_val_data_slice[i]["question"][5:])
  print("ANSWER:  ", get_ground_truths(new_val_data_slice, i)["prediction_text"])
  print()

QUESTION: My insurance ended so what should i do
ANSWER:   Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage , 

QUESTION: Don't do that I'll get insurance 	agent:You will need to get insurance or we will suspend your registration and license 	user:My insurance ended so what should i do
ANSWER:   we mail you an insurance inquiry letter to allow you to clear up the problem. 

QUESTION: I have, that is why I am here to clear that up and change the address on my license and registration 	agent:Okay, have you received a letter from the DMV letting you know how to clear things up? 	user:Don't do that I'll get insurance 	agent:You will need to get insurance or we will suspend your registration and license 	user:My insurance ended so what should i do
ANSWER:   Learn more about ho

In [ ]:
from tqdm import tqdm

preds = []
total_len = new_val_data.num_rows
new_val_data_slice = new_val_data.select(list(range(0, total_len)))

print(total_len)
for i in tqdm(range(len(new_val_data_slice))):
  preds.append(get_ground_truths(new_val_data_slice, i))

preds

3972


100%|██████████| 3972/3972 [00:00<00:00, 4808.34it/s]


[{'id': 'dea7174409afbfe0af0ace21e7f318ae_1',
  'no_answer_probability': 0,
  'prediction_text': 'Because we all pay indirectly for crashes involving uninsured motorists , New York State requires every motorist to maintain auto insurance every single day a vehicle is registered. DMV works with insurance companies to electronically monitor your insurance coverage , '},
 {'id': 'dea7174409afbfe0af0ace21e7f318ae_3',
  'no_answer_probability': 0,
  'prediction_text': 'we mail you an insurance inquiry letter to allow you to clear up the problem. '},
 {'id': 'dea7174409afbfe0af0ace21e7f318ae_5',
  'no_answer_probability': 0,
  'prediction_text': 'Learn more about how to change the address on your license and registrations [1 ] '},
 {'id': 'dea7174409afbfe0af0ace21e7f318ae_7',
  'no_answer_probability': 0,
  'prediction_text': 'you won t learn that you have an insurance problem , and we will suspend your registration and license. Make sure you turn in your vehicle s license plates at DMV befo

In [ ]:
import json

with open('predictions_ground_truth.json', 'w') as outfile:
    json.dump(preds, outfile)